In [3]:
import io
import logging
import opvious
import pandas as pd

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s", datefmt='%H:%M:%S')

client = opvious.Client.from_environment()

In [4]:
demand_df = pd.read_csv(io.StringIO("""
location,tier,product,size,demand
Boston,T1,hoodie,M,50
Boston,T1,shirt,L,30
Boston,T2,shirt,L,25
Boston,T1,shirt,XL,20
Seattle,T1,hoodie,M,100
Seattle,T1,hoodie,L,75
Seattle,T2,hoodie,L,50
Seattle,T1,hoodie,XL,50
Seattle,T1,shirt,L,10
""")).set_index(["location", "product", "size", "tier"])
demand_df

demand
location product size tier        
Boston   hoodie  M    T1        50
         shirt   L    T1        30
                      T2        25
                 XL   T1        20
Seattle  hoodie  M    T1       100
                 L    T1        75
                      T2        50
                 XL   T1        50
         shirt   L    T1        10

In [5]:
supply_df = pd.read_csv(io.StringIO("""
product,size,supply
hoodie,M,100
hoodie,L,50
shirt,L,50
shirt,XL,10
""")).set_index(["product", "size"])
supply_df

supply
product size        
hoodie  M        100
        L         50
shirt   L         50
        XL        10

In [6]:
product_df = pd.read_csv(io.StringIO("""
product,min_allocation,diversity
hoodie,100,2
shirt,10,2
""")).set_index(["product"])
product_df

,min_allocation,diversity
product,,
hoodie,100,2
shirt,10,2


In [10]:
request = await client.prepare_attempt_request(
    formulation_name="product-allocation",
    parameters={
        "demand": demand_df["demand"],
        "demandValue": {"T1": 1, "T2": 0.8},
        "minAllocation": product_df["min_allocation"],
        "productDiversity": product_df["diversity"],
        "supply": supply_df["supply"],
        "maxTotalAllocation": 500
    },
)

In [15]:
attempt = await client.start_attempt(request)
outcome = await client.wait_for_outcome(attempt)
outputs = await client.fetch_attempt_outputs(attempt)

16:59:37 INFO Attempt is running... [elapsed=146 milliseconds]
16:59:38 INFO Attempt is optimal. [elapsed=337 milliseconds, objective=206, gap=0.0%]


In [14]:
outputs.variable("allocation")

value
location product size demandTier       
Seattle  hoodie  M    T1            100
                 L    T1             50
Boston   shirt   L    T1             30
                 XL   T1             10
                 L    T2             20